In [47]:
import torch
import models,settings
import numpy as np

In [106]:
ckpt = torch.load(f"../{settings.MODEL}", map_location="cpu")
clf = models.BowmanEntailmentClassifier
enc = models.TextEncoder(len(ckpt["stoi"]))
model=clf(enc)
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [133]:
prune_metrics_dir = "../models/bowman_snli/6.pth"
weights=torch.load(f"{prune_metrics_dir}")['state_dict']['mlp.0.weight']
prev_total_pruned_amt=torch.where(weights==0,1,0).sum()

model.load_state_dict(weights)

TypeError: Expected state_dict to be dict-like, got <class 'torch.Tensor'>.

In [113]:
weights = model.mlp[0].weight.t().detach().cpu().numpy()

In [114]:
weights

array([[ 0.1020597 ,  0.04020922, -0.18008865, ..., -0.02218907,
        -0.02251127,  0.10539793],
       [ 0.10230313,  0.00156967,  0.10115597, ...,  0.00161468,
         0.03990527, -0.03830835],
       [ 0.04028716, -0.07368106,  0.16273642, ..., -0.07258727,
         0.02142162,  0.01602799],
       ...,
       [ 0.25605595,  0.03776345,  0.07038172, ...,  0.00953927,
         0.01841197, -0.0531592 ],
       [ 0.0333262 ,  0.05475619, -0.11017767, ..., -0.10764154,
        -0.03854869, -0.40262905],
       [ 0.1068556 , -0.00570201, -0.00749541, ..., -0.03318856,
        -0.10414948, -0.05488989]], dtype=float32)

In [130]:
mask=torch.ones(weights.shape)
final_weight=np.sort(np.abs(weights>0)).astype('int')

In [131]:
mask

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [120]:
a=torch.where(torch.abs(torch.tensor(final_weight)) == 1, torch.zeros(mask.shape), mask)

In [125]:
model.mlp[0].weight.t().detach().cpu().copy_(a)

tensor([[ 1.3091,  0.1033, -0.3737,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.2211, -0.5964, -1.6174,  ...,  0.0000,  0.0000,  0.0000],
        [-1.2485,  0.4992, -0.3649,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-1.4813, -0.6652,  0.5357,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2188,  1.4192,  0.1945,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5796, -0.0526,  0.8584,  ...,  0.0000,  0.0000,  0.0000]])

In [129]:
np.round((torch.where(model.mlp[0].weight.t()==0,1,0).sum()/(1024*2048)),1)

tensor(0.5000)